# Visualizing Raw 2-Photon Images

### Environment Setup

In [1]:
from dandi import dandiapi
import dandi
import fsspec
import pynwb
import h5py
from fsspec.implementations.cached import CachingFileSystem
from nwbwidgets import nwb2widget
from matplotlib import pyplot as plt
import numpy as np
from mpl_interactions import ipyplot as iplt

### Streaming 2P File

In [2]:
dandiset_id = "000336"
filepath = "1214523350/1214648740"
authenticate = True
dandi_api_key = "5c1eb544e6870c4c2ea676d22312cefb73e161e1"

In [3]:
if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)

print(f"Got dandiset {my_dandiset}")

A newer version (0.46.6) of dandi/dandi-cli is available. You are using 0.46.3
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
Retrying GET https://api.dandiarchive.org/api/auth/token
HTTP connection failed
Traceback (most recent call last):
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\dandi\dandiapi.py", line 202, in request
    for i, attempt in enumerate(
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\si

HTTPError: 503 Server Error: Service Unavailable for url: https://api.dandiarchive.org/api/auth/token

In [ ]:
file = my_dandiset.get_asset_by_path(filepath)
base_url = file.client.session.head(file.base_download_url)
file_url = base_url.headers['Location']

print(f"Retrieved file url {file_url}")

In [ ]:
fs = CachingFileSystem(
    fs=fsspec.filesystem("http"),
    cache_storage="nwb-cache",  # Local folder for the cache
)

f = fs.open(file_url, "rb")
file = h5py.File(f)
io = pynwb.NWBHDF5IO(file=file, mode='r', load_namespaces=True)
nwb = io.read()

In [ ]:
data = nwb.acquisition["raw_data"].data[0]

In [ ]:
plt.imshow(data, interpolation='nearest')
plt.show()

In [ ]:
%matplotlib ipympl
